In [1]:
try:
    from bifacial_radiance import *
except ImportError:
    raise RuntimeError('bifacial_radiance is required. download distribution')
    # Simple example system using Radiance.
    
import numpy as np

In [4]:
# The below routine creates a HEXAGONAL torque tube, for a 2-UP configuration of a specific module size. Parameters for the module, the torque tube, and the scene are below.
# This is being run with gendaylit, for one specific timestamp
# MODIFY THE TEST FOLDER TO YOUR LIKING. I utmostly dislike the interactive directory.
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Demo3'

timestamp = 4020 # Noon, June 17th. 
simulationname = 'Regular_Array'

# MakeModule Parameters
module_type=simulationname
x = 1.996  # 2-up portrait Longi with 15cm additional gap
y = 0.991
orientation='portrait' 
tilt = 30

# TorqueTube Parameters
torqueTube = True
numpanels = 2
ygap = 0.5
tubetype = 'round'
zgap = 0.30
diameter = 0.2
torqueTubeMaterial = 'Metal_Grey'       # IT's NOT GRAY, IT's GREY
axisofrotationTorqueTube=True
cellLevelModule = True

# SceneDict Parameters
gcr = 0.33   # ground cover ratio,  = module_height / pitch
albedo = 0.28  #'concrete'     # ground albedo
clearance_height = diameter/2.0   
azimuth_ang=90
sensorsy = 200 # Notice we will be doing 200 sensors! It takes more time but gives better resolution.
xgap = 0.5

# Now let's run the example

demo = RadianceObj(simulationname,path = testfolder)  # Create a RadianceObj 'object'
demo.setGround(0.62) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
epwfile = demo.getEPW(37.5,-77.6) # pull TMY data for any global lat/lon
metdata = demo.readEPW(epwfile) # read in the EPW weather data from above
demo.gendaylit(metdata,4020)  # Noon, June 17th

# Making module with all the variables
moduledict=demo.makeModule(name=simulationname,x=x,y=y,bifi=1, 
           torquetube=torqueTube, diameter = diameter, tubetype = tubetype, material = torqueTubeMaterial, zgap = zgap, numpanels = numpanels, ygap = ygap, rewriteModulefile = True, xgap=xgap, axisofrotationTorqueTube=axisofrotationTorqueTube, cellLevelModule=cellLevelModule)
# create a scene with all the variables
sceneDict = {'tilt':tilt,'pitch': np.round(moduledict['sceney'] / gcr,3),'height':clearance_height,'azimuth':azimuth_ang, 'module_type':simulationname}  


scene = demo.makeScene(moduletype=module_type, sceneDict=sceneDict, nMods = 1, nRows = 1, sensorsy = sensorsy) #makeScene creates a .rad file with 20 modules per row, 7 rows.
octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.


path = C:\Users\sayala\Documents\RadianceScenes\Demo3
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw  ... OK!
('\nModule Name:', 'Regular_Array')
REWRITING pre-existing module file. 
Module Regular_Array successfully created
Case 0 
Created Regular_Array.oct


In [5]:
name='Mycube'
text='! genbox black cuteBox 0.05 0.05 2.5 | xform -t -0.025 -0.025 0'
customObject = demo.makeCustomObject(name,text)
demo.appendtoScene(scene.radfile, customObject, '!xform -rz 0')
octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.

('\nCustom Object Name', 'objects\\Mycube.rad')
Created Regular_Array.oct


# At this point you should be able to go into a command window (cmd.exe) and check the geometry. Example:

rvu -vf views\front.vp -e .01 Regular_Array.oct

And then proceed happily with your analysis: